In [ ]:
import pandas as pd
import os
import numpy as np
import pickle
import gc
from sklearn.preprocessing import LabelEncoder
import multiprocessing as mp

In [ ]:
data_path = '../../../data'
features_path = '../1pre_features'
answer_info = pd.read_csv(os.path.join(data_path, 'answer_info_0926.txt'), header=None, sep='\t')
#train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')

#train1.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
answer_info.columns = ['回答id', '问题id', '用户id', '回答创建时间', '回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数', '点赞数', '取赞数', '评论数', '收藏数', '感谢数', '举报数', '没有帮助数', '反对数']

drop_feat = ['回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频',]
answer_info  = answer_info.drop(drop_feat, axis=1)

#user_answer_info = pd.merge(train1, answer_info, how='left', on='用户id')

In [ ]:
answer_info['回答创建时间-hour'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
answer_info['回答创建时间-day'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)

In [ ]:
with open('trainuser_topic.pkl','rb') as file:
    train_user_answered_topics = pickle.load(file)
print(train_user_answered_topics.head())

In [ ]:
answer_info_train = pd.merge(answer_info, train_user_answered_topics, left_on='用户id',right_on='uid', how='left').drop(['uid'], axis=1)
print(answer_info_train.head(5))

In [ ]:
del answer_info_train

In [ ]:
print(1)

In [ ]:
with open('valuser_topic.pkl','rb') as file:
    test_user_answered_topics = pickle.load(file)
print(test_user_answered_topics.head())

In [ ]:
answer_info_test = pd.merge(answer_info, test_user_answered_topics, left_on='用户id',right_on='uid', how='left').drop(['uid'], axis=1)
print(answer_info_test.head(5))

In [ ]:
del answer_info

In [ ]:
# 减少内存占用
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        df[col] = df[col].fillna(0)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int' or str(col_type)[:5] == 'float':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            
#             if str(col_type)[:5] == 'float':
#                 df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# 减少内存占用
def reduce_mem_usage111(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        df[col] = df[col].fillna(0)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':# or str(col_type)[:5] == 'float':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            
            if str(col_type)[:5] == 'float':
                df[col] = df[col].astype(np.float16)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
with open('traintopic_count.pkl','rb') as file:
    topic_count_train = pickle.load(file)
print(train_topic_count.head())

with open('valtopic_count.pkl','rb') as file:
    topic_count_test = pickle.load(file)
print(test_topic_count.head())

train_u_topic_count = pd.merge(train_user_answered_topics, topic_count_train, left_on='u_t_answered_topics', right_on='user_topic_id', how='left').drop(['user_topic_id'], axis=1)
test_u_topic_count = pd.merge(test_user_answered_topics, topic_count_test, left_on='u_t_answered_topics', right_on='user_topic_id', how='left').drop(['user_topic_id'], axis=1)
print(train_u_topic_count.head())

In [ ]:
print(train_u_topic_count.info())

In [ ]:
def merge_u_topic(u_topic_info, etype, func):
    question_label = u_topic_info.copy()
    question_label['topic_'+func] = question_label['topic_'+func].fillna(0)
    question_label['topic_'+func] = question_label['topic_'+func].astype(int)
    question_label['u_t_a_'+func+'_sum'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('sum').astype(int))
    question_label['u_t_a_'+func+'_mean'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('mean'))
    question_label['u_t_a_'+func+'_max'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('max').astype(int))
    question_label['u_t_a_'+func+'_std'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('std'))
    question_label['u_t_a_'+func+'_min'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('min').astype(int))
    
    del question_label['topic_'+func], question_label['u_t_answered_topics']
    question_label = question_label.drop_duplicates(inplace=False)
    
    print(question_label.info())
    
    with open('for_ut/'+etype+'user_topic_'+func+'.pkl', 'wb') as file:
        pickle.dump(question_label, file)
    
    return question_label

In [ ]:
train_u_topic_count_stats = merge_u_topic(train_u_topic_count, 'train', 'count')
test_u_topic_count_stats = merge_u_topic(test_u_topic_count, 'val', 'count')

In [ ]:
def get_t_count(func, train_user_answered_topics, test_user_answered_topics):

    with open('traintopic_'+func+'_count.pkl','rb') as file:
        topic_count_train = pickle.load(file)
    print(topic_count_train.head())

    with open('valtopic_'+func+'_count.pkl','rb') as file:
        topic_count_test = pickle.load(file)
    print(topic_count_test.head())

    train_u_topic_count = pd.merge(train_user_answered_topics, topic_count_train, left_on='u_t_answered_topics', right_on='user_topic_id', how='left').drop(['user_topic_id'], axis=1)
    test_u_topic_count = pd.merge(test_user_answered_topics, topic_count_test, left_on='u_t_answered_topics', right_on='user_topic_id', how='left').drop(['user_topic_id'], axis=1)
    
    print(train_u_topic_count.head())
    return train_u_topic_count, test_u_topic_count

In [ ]:
with open('traintopic_inv_count.pkl','rb') as file:
    topic_count_inv_train = pickle.load(file)
print(train_topic_count.info())

In [ ]:
train_u_topic_good_count, test_u_topic_good_count = get_t_count('good', train_user_answered_topics, test_user_answered_topics)
train_u_topic_good_stats = merge_u_topic(train_u_topic_good_count, 'train', 'good_count')
test_u_topic_good_stats = merge_u_topic(test_u_topic_good_count, 'val', 'good_count')

In [ ]:
train_u_topic_mark_count, test_u_topic_mark_count = get_t_count('mark', train_user_answered_topics, test_user_answered_topics)
train_u_topic_mark_stats = merge_u_topic(train_u_topic_mark_count, 'train', 'mark_count')
test_u_topic_mark_stats = merge_u_topic(test_u_topic_mark_count, 'val', 'mark_count')

In [ ]:
train_u_topic_thank_count, test_u_topic_thank_count = get_t_count('thank', train_user_answered_topics, test_user_answered_topics)
train_u_topic_thank_stats = merge_u_topic(train_u_topic_thank_count, 'train', 'thank_count')
test_u_topic_thank_stats = merge_u_topic(test_u_topic_thank_count, 'val', 'thank_count')

In [ ]:
def get_inv_t_count(func, train_user_answered_topics, test_user_answered_topics):

    with open('traintopic_'+func+'_count.pkl','rb') as file:
        topic_count_train = pickle.load(file)
    print(topic_count_train.head())

    with open('valtopic_'+func+'_count.pkl','rb') as file:
        topic_count_test = pickle.load(file)
    print(topic_count_test.head())

    train_u_topic_count = pd.merge(train_user_answered_topics, topic_count_train, left_on='u_t_answered_topics', right_on='user_inv_topic_id', how='left').drop(['user_inv_topic_id'], axis=1)
    test_u_topic_count = pd.merge(test_user_answered_topics, topic_count_test, left_on='u_t_answered_topics', right_on='user_inv_topic_id', how='left').drop(['user_inv_topic_id'], axis=1)
    
    print(train_u_topic_count.head())
    return train_u_topic_count, test_u_topic_count

In [ ]:
print(train_u_topic_inv_count.info())

In [ ]:
train_u_topic_inv_count, test_u_topic_inv_count = get_inv_t_count('inv', train_user_answered_topics, test_user_answered_topics)

In [ ]:
def merge_u_inv_topic(u_topic_info, etype, func):
    question_label = u_topic_info.copy()
    question_label['topic_'+func] = question_label['topic_'+func].fillna(0)
    question_label['topic_'+func] = question_label['topic_'+func].astype(int)
    question_label['u_t_a_'+func+'_sum'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('sum'))
    question_label['u_t_a_'+func+'_mean'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('mean'))
    question_label['u_t_a_'+func+'_max'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('max'))
    question_label['u_t_a_'+func+'_std'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('std'))
    question_label['u_t_a_'+func+'_min'] = question_label['uid'].map(question_label.groupby('uid')['topic_'+func].agg('min'))
    
    del question_label['topic_'+func], question_label['u_t_answered_topics']
    question_label = question_label.drop_duplicates(inplace=False)
    
    print(question_label.info())
    
    with open('for_ut/'+etype+'user_topic_'+func+'.pkl', 'wb') as file:
        pickle.dump(question_label, file)
    
    return question_label

In [ ]:
train_u_topic_inv_stats1 = merge_u_inv_topic(train_u_topic_inv_count, 'train', 'inv_count')
test_u_topic_inv_stats1 = merge_u_inv_topic(test_u_topic_inv_count, 'val', 'inv_count')

train_u_topic_inv_stats2 = merge_u_inv_topic(train_u_topic_inv_count, 'train', 'inv_std')
test_u_topic_inv_stats2 = merge_u_inv_topic(test_u_topic_inv_count, 'val', 'inv_std')

train_u_topic_inv_stats3 = merge_u_inv_topic(train_u_topic_inv_count, 'train', 'inv_sum')
test_u_topic_inv_stats3 = merge_u_inv_topic(test_u_topic_inv_count, 'val', 'inv_sum')

train_u_topic_inv_stats4 = merge_u_inv_topic(train_u_topic_inv_count, 'train', 'inv_pos_mean')
test_u_topic_inv_stats4 = merge_u_inv_topic(test_u_topic_inv_count, 'val', 'inv_pos_mean')

train_u_topic_inv_stats5 = merge_u_inv_topic(train_u_topic_inv_count, 'train', 'inv_neg_mean')
test_u_topic_inv_stats5 = merge_u_inv_topic(test_u_topic_inv_count, 'val', 'inv_neg_mean')

In [ ]:
print(test_u_topic_inv_stats4[test_u_topic_inv_stats4['u_t_a_inv_pos_mean_min']!=0])

### -----------总结-----------

### 离线加载

In [ ]:
with open('for_ut/trainuser_topic_count.pkl','rb') as file:
    train_u_topic_count_stats = pickle.load(file)
    
with open('for_ut/trainuser_topic_good_count.pkl','rb') as file:
    train_u_topic_good_stats = pickle.load(file)
    
with open('for_ut/trainuser_topic_mark_count.pkl','rb') as file:
    train_u_topic_mark_stats = pickle.load(file)
    
with open('for_ut/trainuser_topic_thank_count.pkl','rb') as file:
    train_u_topic_thank_stats = pickle.load(file)
    
with open('for_ut/trainuser_topic_inv_pos_mean.pkl','rb') as file:
    train_u_topic_inv_stats4 = pickle.load(file)
    
with open('for_ut/trainuser_topic_inv_sum.pkl','rb') as file:
    train_u_topic_inv_stats3 = pickle.load(file)


In [ ]:
print(test_u_topic_count_stats.info())

In [ ]:
print(train_u_topic_inv_stats4.head(30))

In [ ]:
with open('for_ut/valuser_topic_count.pkl','rb') as file:
    test_u_topic_count_stats = pickle.load(file)
    
with open('for_ut/valuser_topic_good_count.pkl','rb') as file:
    test_u_topic_good_stats = pickle.load(file)
    
with open('for_ut/valuser_topic_mark_count.pkl','rb') as file:
    test_u_topic_mark_stats = pickle.load(file)
    
with open('for_ut/valuser_topic_thank_count.pkl','rb') as file:
    test_u_topic_thank_stats = pickle.load(file)
    
with open('for_ut/valuser_topic_inv_pos_mean.pkl','rb') as file:
    test_u_topic_inv_stats4 = pickle.load(file)
    
with open('for_ut/valuser_topic_inv_sum.pkl','rb') as file:
    test_u_topic_inv_stats3 = pickle.load(file)

In [ ]:
train_u_topic_count_stats = reduce_mem_usage(train_u_topic_count_stats)
train_u_topic_good_stats = reduce_mem_usage(train_u_topic_good_stats)
train_u_topic_thank_stats = reduce_mem_usage(train_u_topic_thank_stats)
train_u_topic_mark_stats = reduce_mem_usage(train_u_topic_mark_stats)

In [ ]:
#train_u_topic_inv_stats1 = reduce_mem_usage(train_u_topic_inv_stats1)
#train_u_topic_inv_stats2 = reduce_mem_usage(train_u_topic_inv_stats2)
train_u_topic_inv_stats3 = reduce_mem_usage111(train_u_topic_inv_stats3)
train_u_topic_inv_stats4 = reduce_mem_usage111(train_u_topic_inv_stats4)
#train_u_topic_inv_stats5 = reduce_mem_usage(train_u_topic_inv_stats5)

In [ ]:
list(train_u_a_topic.columns.values)

In [ ]:
train_u_a_topic = pd.merge(train_u_topic_count_stats, train_u_topic_good_stats, on='uid', how='left')
#train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_good_stats, on='uid', how='outer')
train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_mark_stats, on='uid', how='left')
train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_thank_stats, on='uid', how='left')
#train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_inv_stats1, on='uid', how='left)
#train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_inv_stats2, on='uid', how='left')
#train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_inv_stats3, on='uid', how='left')
#train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_inv_stats4, on='uid', how='left')
#train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_inv_stats5, on='uid', how='left')

In [ ]:
train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_inv_stats4, on='uid', how='left')

In [ ]:
train_u_a_topic = train_u_a_topic.drop(['topic_inv_neg_mean','topic_inv_sum','topic_inv_std','topic_inv_count'], axis=1,  inplace=False)

In [ ]:
print(train_u_a_topic.head(20))

In [ ]:
test_u_topic_count_stats = reduce_mem_usage(test_u_topic_count_stats)
test_u_topic_good_stats = reduce_mem_usage(test_u_topic_good_stats)
test_u_topic_thank_stats = reduce_mem_usage(test_u_topic_thank_stats)
test_u_topic_mark_stats = reduce_mem_usage(test_u_topic_mark_stats)
#test_u_topic_inv_stats1 = reduce_mem_usage(test_u_topic_inv_stats1)
#test_u_topic_inv_stats2 = reduce_mem_usage(test_u_topic_inv_stats2)
#test_u_topic_inv_stats3 = reduce_mem_usage(test_u_topic_inv_stats3)
#test_u_topic_inv_stats4 = reduce_mem_usage(test_u_topic_inv_stats4)
#test_u_topic_inv_stats5 = reduce_mem_usage(test_u_topic_inv_stats5)

In [ ]:
test_u_topic_inv_stats3 = reduce_mem_usage111(test_u_topic_inv_stats3)
test_u_topic_inv_stats4 = reduce_mem_usage111(test_u_topic_inv_stats4)

In [ ]:
print(1)

In [ ]:
test_u_a_topic = pd.merge(test_u_topic_count_stats, test_u_topic_good_stats, on='uid', how='outer')
#test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_good_stats, on='uid', how='outer')
test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_mark_stats, on='uid', how='outer')
test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_thank_stats, on='uid', how='outer')
#test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_inv_stats1, on='uid', how='outer')
#test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_inv_stats2, on='uid', how='outer')
#test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_inv_stats3, on='uid', how='outer')
test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_inv_stats4, on='uid', how='outer')
#test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_inv_stats5, on='uid', how='outer')

In [ ]:
test_u_a_topic = test_u_a_topic.drop(['topic_inv_neg_mean','topic_inv_sum','topic_inv_std','topic_inv_count'], axis=1,  inplace=False)

### 用户和话题基本的交叉

#### 用户回答过的话题个数

In [ ]:
print(train_user_answered_topics.info())

In [ ]:
train_user_answered_topics['u_t_a_num'] = train_user_answered_topics['uid'].map(train_user_answered_topics['uid'].value_counts().astype(int))
test_user_answered_topics['u_t_a_num'] = test_user_answered_topics['uid'].map(test_user_answered_topics['uid'].value_counts().astype(int))

In [ ]:
train_u_topic_num = train_user_answered_topics[['uid','u_t_a_num']].drop_duplicates(inplace=False)
test_u_topic_num = test_user_answered_topics[['uid','u_t_a_num']].drop_duplicates(inplace=False)

In [ ]:
train_u_a_topic = pd.merge(train_u_a_topic, train_u_topic_num, on='uid', how='left')
test_u_a_topic = pd.merge(test_u_a_topic, test_u_topic_num, on='uid', how='left').drop_duplicates(inplace=False).info())


In [ ]:
train_u_a_topic = train_u_a_topic.drop_duplicates(inplace=False)
test_u_a_topic = test_u_a_topic.drop_duplicates(inplace=False)

In [ ]:
with open('for_ut/user_topic_all_count_train.pkl', 'wb') as file:
    pickle.dump(train_u_a_topic, file)

with open('for_ut/user_topic_all_count_test.pkl', 'wb') as file:
    pickle.dump(test_u_a_topic, file)

In [ ]:
print(train_u_a_topic.drop_duplicates(inplace=False).info())

#### 用户回答过的问题个数除以回答过的话题个数